In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

2024-02-09 22:57:21.978877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("cleaned_data.csv")
df = df.sample(frac=1)
df['gender'].replace(["Male", "Female"], [0, 1], inplace=True)

In [3]:
cc_cols = [x for x in df.columns if x[:3] == "cc_"]
med_cols = [x for x in df.columns if x[:5] == "meds_"]
with open("pmh_cols.txt") as f:
    pmh_cols = f.readlines()
pmh_cols = list(map(lambda x:x[:-1], pmh_cols))

x_cols = ["age", "gender"] + cc_cols + pmh_cols
y_cols = ["disposition"] + med_cols

esi_data = pd.get_dummies(df['esi'], dtype=int)

In [4]:
val_split = 0.9

np_x_train = np.array(df[x_cols])[:int(val_split * df.shape[0]), :]
np_esi_train = np.array(esi_data)[:int(val_split * df.shape[0]), :]

np_x_val = np.array(df[x_cols])[int(val_split * df.shape[0]):, :]
np_esi_val = np.array(esi_data)[int(val_split * df.shape[0]):, :]

print(np_x_train.shape)
print(np_esi_train.shape)
print()
print(np_x_val.shape)
print(np_esi_val.shape)

(502216, 483)
(502216, 5)

(55802, 483)
(55802, 5)


In [5]:
gen_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(49, activation="softmax")
])

gen_model.compile(
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(curve="PR"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [15]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01, decay_steps=5, decay_rate=0.99, staircase=True
)

esi_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])

esi_model.compile(
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

In [16]:
esi_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 200)               96800     
                                                                 
 dense_10 (Dense)            (None, 100)               20100     
                                                                 
 dense_11 (Dense)            (None, 5)                 505       
                                                                 
Total params: 117405 (458.61 KB)
Trainable params: 117405 (458.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
esi_model.fit(
    x=np_x_train,
    y=np_esi_train,
    epochs=10,
    validation_data=(np_x_val, np_esi_val),
    batch_size=16,
)

Epoch 1/10
31389/31389 [==============================] - 86s 3ms/step - loss: 0.8740 - accuracy: 0.6316 - val_loss: 0.8505 - val_accuracy: 0.6419
Epoch 2/10
31389/31389 [==============================] - 79s 3ms/step - loss: 0.8557 - accuracy: 0.6385 - val_loss: 0.8478 - val_accuracy: 0.6435
Epoch 3/10
31389/31389 [==============================] - 76s 2ms/step - loss: 0.8537 - accuracy: 0.6401 - val_loss: 0.8479 - val_accuracy: 0.6417
Epoch 4/10
31389/31389 [==============================] - 111s 4ms/step - loss: 0.8515 - accuracy: 0.6414 - val_loss: 0.8512 - val_accuracy: 0.6416
Epoch 5/10
 7520/31389 [======>.......................] - ETA: 1:33 - loss: 0.8508 - accuracy: 0.6415

KeyboardInterrupt: 